In [7]:
from ionqvision.ansatze.ansatz_library import AngleEncoder
encoder = AngleEncoder(num_qubits=4)

from ionqvision.ansatze.ansatz_library import QCNNAnsatz
ansatz = QCNNAnsatz(num_qubits=4)

from qiskit.quantum_info import SparsePauliOp

# Measure the expectation value of X_0, Y_0, Z_0
quantum_features = [
    SparsePauliOp(["IIIX"]), 
    SparsePauliOp(["IIIY"]), 
    SparsePauliOp(["IIIZ"])
]

In [8]:
from ionqvision.modules import BinaryMNISTClassifier
classifier = BinaryMNISTClassifier(encoder, ansatz, quantum_features); classifier

INFO:qiskit.passmanager.base_tasks:Pass: ContainsInstruction - 0.01073 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.01812 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.03266 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.07319 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: ElidePermutations - 0.00787 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: RemoveDiagonalGatesBeforeMeasure - 0.03290 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.10276 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: CommutationAnalysis - 0.22626 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: CommutativeCancellation - 0.19860 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Collect2qBlocks - 0.05889 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: ConsolidateBlocks - 0.41485 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Split2QUnitaries - 0.06461 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00930 (ms)
INFO:qiskit.passm

BinaryMNISTClassifier(
  (latent_vec_encoder): Sequential(
    (0): Linear(in_features=168, out_features=4, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Sigmoid()
  )
  (quantum_layer): QuantumModule()
  (prediction_head): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Sigmoid()
  )
)

In [12]:
%%time
# Get a (pre-processed) training and test set
train_set, test_set = classifier.get_train_test_set(train_size=4000, test_size=100)

# Configure model training hyperparameters
config = {
    "epochs": 8,
    "lr": 0.01,
    "batch_size": 400,
    "betas": (0.9, 0.99),
    "weight_decay": 5e-4,
    "clip_grad": True,
    "log_interval": 1,
}

# Train and plot the results
classifier.train_module(train_set, test_set, config)
classifier.plot_training_progress()

epoch:   9 | loss: 0.718
lr: 0.0100 | processed     1/   10 batches per epoch in 641.23s (11.20s forward / 442.77s backward)
Model achieved 42.275%  accuracy on TRAIN set.
Model achieved 47.000%  accuracy on TEST set.

epoch:   9 | loss: 0.701
lr: 0.0100 | processed     2/   10 batches per epoch in 1325.90s (20.73s forward / 475.21s backward)
Model achieved 41.625%  accuracy on TRAIN set.
Model achieved 42.000%  accuracy on TEST set.



KeyboardInterrupt: 

In [ ]:
from qbraid.runtime import QbraidSession
from ionqvision.scq_client import IonQSCQClient

session = QbraidSession()
client = IonQSCQClient(session=session)

submission_data = classifier.submit_model_for_grading(client=client)
message = submission_data.get("message")
submission_id = submission_data.get("submissionId")

print(message)

client.await_autograder_result(submission_id=submission_id)